In [1]:
import sys
import os
import numpy as np
from _3dpu_using_dfs import *
ROOT = "../"

In [2]:
import yaml

# Load the YAML file
with open(ROOT + 'paths.yaml', 'r') as file:
    config = yaml.safe_load(file)

# Access paths
data_path = config['paths']["data_big"]

In [3]:
import nibabel as nb
t = 3
data = nb.load(data_path).get_fdata()
data = np.array(data)[:,:,:,t]

In [6]:
def iterative_dfs(C, start):
    stack = [(start, [start])]
    C.visited[start] = True
    path_dic = {start: 0}
    while stack:
        v, path = stack.pop()
        C.visited[v] = True
        path_dic[v] = len(path) - 1
        print(stack)
        for neighbour in C.Res_graph[v]:
            if not C.visited[neighbour]:
                newPath = path + [neighbour]
                stack.append((neighbour, newPath))
            elif neighbour in path:
                cycle_start_index = path_dic[neighbour]
                cycle = path[cycle_start_index:] + [neighbour]

                C.cycles.append(cycle)
                for point in cycle:
                    C.visited[point] = True
                    C.incycles[point] = True
                return cycle

    return stack

def detect_cycles_iterative(C):
    for i in tqdm(range(len(C.mapping))):
        if not C.visited[i]:
            iterative_dfs(C, i)
                

In [4]:
C = Resiuals(data)
C.map_nodes()
C.create_graph()
C.cycles = []
C.incycles = [1]*len(C.mapping)
C.connex =[1]*len(C.mapping)
C.visited = [False]*len(C.mapping)
# iterative_dfs(C,6378)

100%|██████████| 2023266/2023266 [00:10<00:00, 195088.27it/s]


In [6]:
triples = [value for key,value in C.inverted_dictionnary.items() if len(key) == 3]
doubles = [value for key,value in C.inverted_dictionnary.items() if len(key) == 2]

In [7]:
for double in tqdm(doubles):
    assert len(C.Res_graph[double[0]]) == 2, "Error this one has not two"
    C.Res_graph[double[0]] =  C.Res_graph[double[0]][:1]
    C.Res_graph[double[1]] =  C.Res_graph[double[1]][1:]


100%|██████████| 296508/296508 [00:00<00:00, 452240.02it/s]


In [8]:
for triple in tqdm(triples):
    assert(len(C.Res_graph[triple[0]]) == 3), "Error this has not three"
    C.Res_graph[triple[0]] = [C.Res_graph[triple[0]][0]]
    C.Res_graph[triple[1]] = [C.Res_graph[triple[1]][1]]
    C.Res_graph[triple[2]] = [C.Res_graph[triple[2]][2]]

100%|██████████| 3924/3924 [00:00<00:00, 269404.32it/s]


In [128]:
C.Res_graph[triples[6][0]]

[620242]

In [9]:
C.fill_open_paths()

100%|██████████| 2023266/2023266 [00:01<00:00, 1762048.44it/s]


{1933316, 1998855, 1343496, 1343497, 1703945, 1638411, 1671179, 1867789, 1835026, 1703955, 1769493, 1900568, 1900569, 1409059, 1867816, 1703977, 1802286, 1966130, 1638454, 1572919, 1638455, 1769542, 1900615, 1704015, 1769562, 1704027, 1966172, 1704032, 1474659, 1867875, 1638501, 1671269, 1376361, 1769578, 1540203, 1704043, 1769592, 1769593, 1572986, 1791071, 1867906, 1605775, 1441937, 1856204, 1573014, 1638553, 1802395, 1376415, 1704099, 1804104, 1736871, 1638576, 1605809, 1900724, 1540279, 1671357, 1802438, 950472, 950473, 950475, 950476, 1704139, 950478, 950480, 950481, 950482, 950484, 950486, 950488, 950489, 950490, 1573083, 1704155, 950493, 1376477, 950495, 1540319, 1605855, 1343714, 1966298, 1736932, 1114343, 1114345, 1114347, 1769707, 1999083, 1999084, 1114351, 1114353, 1442034, 1114355, 1868020, 1868021, 1114358, 1114359, 1769720, 1114361, 1605882, 1114364, 1114365, 1507580, 1114368, 1671424, 1114371, 1114373, 1114375, 1114378, 1114383, 1114385, 1114386, 1114387, 1376537, 176975

100%|██████████| 16578/16578 [00:00<00:00, 61134.35it/s]


16578
16576
16350
12262
10373
8898
8188
7358
6771
6195
5773
5351
5072
4737
4512
4304
4099
3865
3698
3515
3373
3241
3119
2987
2870
2750
2668
2579
2497
2407
2337
2241
2162
2106
2051
1975
1929
1882
1835
1776
1717
1664
1615
1570
1527
1490
1448
1405
1368
1323
1285
1259
1235
1202
1167
1137
1120
1096
1064
1042
1028
1002
980
959
942
921
899
882
867
848
827
814
800
787
756
738
726
708
692
676
657
647
637
623
607
591
579
567
556
544
536
522
512
503
493
487
475
465
456
450
441
433
425
419
413
401
392
386
379
368
360
355
348
342
335
326
322
315
310
303
297
292
288
284
280
277
274
272
265
262
257
255
249
244
241
236
234
230
227
225
221
215
210
206
202
197
194
190
189
186
184
183
180
177
174
168
167
166
164
160
157
155
154
151
149
148
147
144
140
138
136
130
127
126
125
124
122
118
115
112
110
107
106
103
101
100
99
98
97
97
97
95
93
92
91
91
91
88
86
85
84
83
82
82
82
82
81
80
79
78
77
77
76
75
73
72
71
70
69
69
68
68
68
68
67
66
66
65
65
64
61
60
60
59
57
55
54
53
52
51
51
51
50
49
48
47
47
47
47


In [11]:
Points = set()
D = []
for op in C.open_paths:
    for pt in op:
        Points.add(pt)

In [12]:
Open_path_points = list()
for op in C.open_paths:
    for k in op:
        Open_path_points.append(k)
cycle_path_points = set(list(C.Res_graph.keys()))
cycle_path_points.difference_update(Open_path_points)

In [146]:
len(set(Open_path_points)) == len(Open_path_points)

True

In [13]:
Edning_points = list()
ending_paths = list()
for k in C.Res_graph:
    if C.Res_graph[k] == []:
        Edning_points.append(k)
bad_one = None
visited = set()
for op in C.open_paths:
    visited.update(op)
    if  op[-1] not in Edning_points:

AssertionError: [1638454, 257342, 1640570, 933128, 257353, 1638473, 255380, 1636300, 255379, 1638472, 930878, 257366, 933150, 933127, 259268, 935361, 1642659, 1642658, 935372, 1642675, 935387, 1642689, 935407, 259316, 1640630, 933183, 1640618, 933184, 933196, 259334, 935438, 935461, 259369, 933243, 257433, 1638529, 255440, 928527, 253444]

In [168]:
L = [1638454, 257342, 1640570, 933128, 257353, 1638473, 255380, 1636300, 255379, 1638472, 930878, 257366, 933150, 933127, 259268, 935361, 1642659, 1642658, 935372, 1642675, 935387, 1642689, 935407, 259316, 1640630, 933183, 1640618, 933184, 933196, 259334, 935438, 935461, 259369, 933243, 257433, 1638529, 255440, 928527, 253444]

In [21]:
point = 253444
[n for n in C.Res_graph[point] if (n not in visited)]

[926065]

In [161]:
set(Edning_points) == set()

True

In [149]:
len(C.open_paths)

16578

In [132]:
for k in cycle_path_points:
    assert len(C.Res_graph[k]) == 1, k

AssertionError: 606345

In [54]:
A = (list(range(5)))
# for k in A:
#     A.pop()
A

[0, 1, 2, 3, 4]

In [61]:
K = 0
for k in C.Res_graph:
    if C.Res_graph[k] == []:
        K += 1

In [143]:
K

In [56]:
layer = set(C.Res_graph.keys())
visited = set()
cycles = []
while layer :
    print(len(layer))
    init = layer.pop()
    cycle = [init]
    next_node = C.Res_graph[init][0]
    while next_node != init:
        cycle.append(next_node)
        assert next_node in layer, next_node
        layer.remove(next_node)
        next_node = C.Res_graph[next_node][0]
    cycle.append(init)
    cycles.append(cycle)
    

1948224


AssertionError: 6214

In [8]:
def tarjan_scc_iterative(graph):
    num_vertices = len(graph)
    index = 0
    indices = [-1] * num_vertices
    low_links = [-1] * num_vertices
    on_stack = [False] * num_vertices
    stack = []
    sccs = []

    # The stack for iterative DFS, items are (node, next_child_idx)
    dfs_stack = []

    for start_vertex in tqdm(range(num_vertices)):
        if indices[start_vertex] == -1:
            dfs_stack.append((start_vertex, 0))

            while dfs_stack:
                node, next_child_idx = dfs_stack[-1]

                if indices[node] == -1:
                    indices[node] = index
                    low_links[node] = index
                    index += 1
                    stack.append(node)
                    on_stack[node] = True

                exhausted = True
                for i in range(next_child_idx, len(graph[node])):
                    neighbor = graph[node][i]
                    if indices[neighbor] == -1:
                        dfs_stack[-1] = (node, i + 1)
                        dfs_stack.append((neighbor, 0))
                        exhausted = False
                        break
                    elif on_stack[neighbor]:
                        low_links[node] = min(low_links[node], indices[neighbor])

                if exhausted:
                    if low_links[node] == indices[node]:
                        scc = []
                        while stack[-1] != node:
                            w = stack.pop()
                            on_stack[w] = False
                            scc.append(w)
                        scc.append(stack.pop())
                        on_stack[node] = False
                        sccs.append(scc)

                    dfs_stack.pop()

    return sccs

sccs = tarjan_scc_iterative(C.Res_graph)

100%|██████████| 2023266/2023266 [00:07<00:00, 279036.90it/s] 


In [12]:
max([len(k) for k in sccs])

11

In [23]:
def tarjanSCC(C):
    index = [None] * len(C.mapping)
    lowlink = [None] * len(C.mapping)
    curr_index = 0
    stack = []
    on_stack = [False] * len(C.mapping)
    scc_list = []

    for start_node in tqdm(range(len(C.mapping))):
        if index[start_node] is None:
            stack.append((start_node, start_node, False))
            while stack:
                node, parent, isReturning = stack.pop()
                if not isReturning:
                    if index[node] is None:
                        index[node] = curr_index
                        lowlink[node] = curr_index
                        curr_index += 1
                        stack.append((node, parent, True))
                        on_stack[node] = True
                        stack.extend((neighbor, node, False) for neighbor in C.Res_graph[node])
                else:
                    if node != start_node:
                        lowlink[parent] = min(lowlink[parent], lowlink[node])
                    if node == parent:
                        if on_stack[node]:
                            scc = []
                            while stack and stack[-1][0] != node:
                                n, _, _ = stack.pop()
                                on_stack[n] = False
                                scc.append(n)
                            scc.append(node)
                            on_stack[node] = False
                            scc_list.append(scc)

    return scc_list

In [24]:
K = tarjanSCC(C)

  0%|          | 0/2023266 [00:00<?, ?it/s]

100%|██████████| 2023266/2023266 [00:05<00:00, 357655.07it/s] 


In [15]:
C.incycles = [False]*len(C.mapping)
C.open_paths = []
C.starting_open_paths = []
C.cycles = []
C.fill_starting_open_paths()
for path in C.cycles:
    for point in path:
        C.incycles[point] = True


  0%|          | 0/2023266 [00:00<?, ?it/s]

100%|██████████| 2023266/2023266 [00:01<00:00, 1981598.29it/s]


In [17]:
C.Res_graph = dict()
for res in C.list_res:
    C.Res_graph[C.res_ordre[res]] = []
for res in tqdm(C.list_res):
    C.nodes_of_not_boundary(res)

100%|██████████| 2023266/2023266 [00:10<00:00, 189441.02it/s]


In [23]:
C.fill_open_paths()

100%|██████████| 2023266/2023266 [00:01<00:00, 1621269.11it/s]


{1933316, 1998855, 1343496, 1343497, 1703945, 1638411, 1671179, 1867789, 1835026, 1703955, 1769493, 1900568, 1900569, 1409059, 1867816, 1703977, 1802286, 1966130, 1638454, 1572919, 1638455, 1769542, 1900615, 1704015, 1769562, 1704027, 1966172, 1704032, 1474659, 1867875, 1638501, 1671269, 1376361, 1769578, 1540203, 1704043, 1769592, 1769593, 1572986, 1791071, 1867906, 1605775, 1441937, 1856204, 1573014, 1638553, 1802395, 1376415, 1704099, 1804104, 1736871, 1638576, 1605809, 1900724, 1540279, 1671357, 1802438, 950472, 950473, 950475, 950476, 1704139, 950478, 950480, 950481, 950482, 950484, 950486, 950488, 950489, 950490, 1573083, 1704155, 950493, 1376477, 950495, 1540319, 1605855, 1343714, 1966298, 1736932, 1114343, 1114345, 1114347, 1769707, 1999083, 1999084, 1114351, 1114353, 1442034, 1114355, 1868020, 1868021, 1114358, 1114359, 1769720, 1114361, 1605882, 1114364, 1114365, 1507580, 1114368, 1671424, 1114371, 1114373, 1114375, 1114378, 1114383, 1114385, 1114386, 1114387, 1376537, 176975

100%|██████████| 16578/16578 [00:00<00:00, 562843.89it/s]


1844
1259
1015
874
766
684
612
530
465
401
349
299
251
225
188
171
157
145
139
129
124
122
118
114
110
107
102
99
95
91
85
81
78
71
63
60
55
51
44
40
35
31
26
26
23
17
14
13
12
12
11
10
9
8
8
8
8
8
8
7
7
7
6
5
5
4
3
3
3
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [77]:
C.open_paths = []
def optimized_dfs(C):
    
    for start_node in tqdm(C.starting_open_paths):
        if start_node not in C.visited:
            C.visited.add(start_node)
            node = start_node
            path = []
            while not C.end_of_path[node]:
                path.append(node)
                next_nodes = [n for n in C.Res_graph[node] if not C.incycles[n]]
                node = next_nodes[0]
            
            C.open_paths.append(path)

                

    return C.open_paths

def fill_open_paths(C):
    antecedant = dict()
    visited_edges = set()
    
    C.fill_starting_open_paths()
    paths = dict()
    layer = set(C.starting_open_paths)
    K = []
    for point in tqdm(layer):
        paths[point] = [point]
        antecedant[point] = point

    while layer != {-1}:
        # cpt += 1
        # if cpt % 100 == 0:
        #     print(len(layer))
        new_layer = set()
        print(len(layer))
        for point in (layer):
            if point != -1 :
                next_nodes = [n for n in C.Res_graph[point] if ((min(n,point),max(n,point)) not in visited_edges )]
                if next_nodes:
                    
                    next_node = next_nodes[0]
                    assert (min(next_node,point),max(next_node,point)) not in visited_edges, "Error we have already visited said point"
                    antecedant[next_node] = antecedant[point]
                    new_layer.add(next_node)
                    paths[antecedant[point]].append(next_node)
                    C.incycles[next_node] = True
                    visited_edges.add((min(point,next_node),max(point,next_node)))
                    

                else:
                    new_layer.add(-1)

        assert layer != new_layer, ("there is a repetition",layer,new_layer)
        layer = new_layer
        

    C.open_paths = list(paths.values())

    


In [137]:
def tarjan_scc(graph):
    num_vertices = len(graph)
    index = 0
    indices = [-1] * num_vertices
    low_links = [-1] * num_vertices
    on_stack = [False] * num_vertices
    stack = []
    sccs = []

    def strongconnect(v):
        nonlocal index
        indices[v] = index
        low_links[v] = index
        index += 1
        stack.append(v)
        on_stack[v] = True

        for w in graph[v]:
            if indices[w] == -1:
                strongconnect(w)
                low_links[v] = min(low_links[v], low_links[w])
            elif on_stack[w]:
                low_links[v] = min(low_links[v], indices[w])

        if low_links[v] == indices[v]:
            scc = []
            while True:
                w = stack.pop()
                on_stack[w] = False
                scc.append(w)
                if w == v:
                    break
            sccs.append(scc)

    for v in range(num_vertices):
        if indices[v] == -1:
            strongconnect(v)

    return sccs


In [132]:
L = tarjan_scc(C.Res_graph)

In [4]:
M = Resiuals(np.empty((254,25,25)))
M.Res_graph[0] = []
M.Res_graph[1] = [2,5]
M.Res_graph[2] = [3]
M.Res_graph[3] = [9,4]
M.Res_graph[4] = [1]
M.Res_graph[5] = [6]
M.Res_graph[6] = [7]
M.Res_graph[7] = [8]
M.Res_graph[8] = [3]
M.Res_graph[9] = [10]
M.Res_graph[10] = [1]
M.incycles = [False]*len(M.Res_graph)

In [21]:
D = Resiuals(np.empty((25,36,467)))
D.Res_graph[0] = [1]
D.Res_graph[1] = [2]
D.Res_graph[2] = [0]
D.Res_graph[3] = [7,4]
D.Res_graph[4] = [5]
D.Res_graph[5] = [0,6]
D.Res_graph[6] = [4,2,0]
D.Res_graph[7] = [5,3]
D.incycles = [False]*len(D.Res_graph)


In [12]:


def explore(node,id_,visited,id_dict):
    stack = []
    visited.add(node)
    stack.append((-1,node))
    unilateral_stack = []
    for _,node in stack:
        unilateral_stack.append(node)

    Advance = True
    while Advance:
        id_dict[node] = id_
        id_ += 1
        next_nodes = [n for n in D.Res_graph[node] if n not in unilateral_stack]
        if next_nodes:
            next_node = next_nodes[0]
            visited.add(next_node)
            id_dict[next_node] = id_
            print(next_node,id_)
            stack.append((node,next_node))
            unilateral_stack.append(next_node)
            node = next_node
        else : 
            print("next step")
            Advance  = False
            supposed_next_node = [n for n in D.Res_graph[node]][0]
            print((node,supposed_next_node))
            stack.append((node,supposed_next_node))
            # back track:
            # if supposed_next_node in unilateral_stack:
            #     indice = unilateral_stack.index(supposed_next_node)
            #     for k in 
            #     parent,son = stack.pop()
            #     id_dict[parent] = min(id_dict[son],id_dict[parent])
    return id_dict,stack,unilateral_stack


id_dict,stack,uni = explore(1,0,set(),dict())

2 1
3 2
9 3
10 4
next step
(10, 1)


In [13]:
node  = 3
id_dict = dict()
low_dict = dict()
id_ = 0
stack = [(node,-1)]
unilateral_stack = [node]
id_dict[node] = id_
low_dict[node] = id_
visited = set()
visited.add(node)
stop = False
while not stop:
    next_nodes = [n for n in D.Res_graph[node] while n not in visited]
    if not next_nodes:
        stop = True
    else:
        id_ = id_ + 1
        next_node = next_nodes[0]
        stack.append((next_node,node))
        unilateral_stack.append(next_node)
        id_dict[next_node] = id_
        low_dict[next_node] = id_

# On cherche le premier next node not in the stack so we can unstack:
last_node = [n for n in D.Res_graph[node] if n not in unilateral_stack][0]




    




{1: 0, 2: 1, 3: 2, 9: 3, 10: 4}


In [25]:
def scc2(graph):
    result = []
    stack = []
    low = {}
    call_stack = []
    for v in tqdm(graph):
        call_stack.append((v, 0, len(low)))
        while call_stack:
            v, pi, num = call_stack.pop()
            if pi == 0:
                if v in low: continue
                low[v] = num
                stack.append(v)
            if pi > 0:
                low[v] = min(low[v], low[graph[v][pi-1]])
            if pi < len(graph[v]):
                call_stack.append((v, pi+1, num))
                call_stack.append((graph[v][pi], 0, len(low)))
                continue
            if num == low[v]:
                comp = []
                while True:
                    comp.append(stack.pop())
                    low[comp[-1]] = len(graph)
                    if comp[-1] == v: break
                result.append(comp)
    return result




In [27]:
R = scc2(C.Res_graph)

  0%|          | 0/2023266 [00:00<?, ?it/s]

100%|██████████| 2023266/2023266 [00:06<00:00, 297173.03it/s] 


In [35]:
Lengths = [len(l) for l in R]
sorted(Lengths,reverse = False)
import matplotlib.pyplot as plt
K = [n for n in Lengths if n != 1]

In [42]:
sorted(K)[-10:]
index


[234, 238, 245, 314, 327, 340, 368, 441, 454, 1740947]

In [44]:
for c in C.Res_graph:
    if tuple(sorted(C.Res_graph[c])) == tuple(sorted(C.Res_graph[1])) :
        print(c)

1
612523


In [92]:
max([len(path) for path in C.open_paths])

10531

In [80]:
Points = set()
C.incycles = [False]* len(C.mapping)
for op in C.open_paths:
    
    for k in op:
        Points.add(k)
        C.incycles[k] = 1 - C.incycles[k]
len(Points)/len(C.mapping)

0.10007186400601799

In [86]:
K = 0
for i,op in enumerate(C.open_paths):
    if len(op) != len(set(op)):
        print(op,i)
        K += 1


[1343496, 4737, 616341, 616311, 4671, 10916, 1349914, 10915, 1343430, 1343429, 1343428, 622718, 1343459, 10941, 1349946, 629473, 1349912, 10914, 1343428] 2
[1638411, 257306, 933076, 259213, 935307, 259223, 933076] 5
[1703955, 1703954, 329256, 1703030, 329255, 1703953, 329256] 9
[1409059, 66701, 684352, 1402526, 684312, 1402486, 684287, 1402460, 60504, 677427, 60534, 1402486] 13
[1966130, 1274840, 1274807, 1966060, 559233, 565298, 1979325, 565297, 1972659, 559231, 1274806, 1274839, 559298, 1281630, 1281591, 559231, 1966059, 559232, 1972659, 565296, 1288529, 1979363, 565335, 1281629, 559297, 553215, 1959474, 553214, 1274838, 559266, 1281590, 559230, 1274773, 553126, 1959381, 547029, 1261059] 17
[1638454, 1638453, 255356, 1636278, 255355, 1638452, 255356, 1636279, 255357, 257342, 1640570] 18
[1900615, 496537, 1200258, 491140, 1193057, 1886525, 485851, 1879344, 480607, 1872272, 1178572, 480582, 1185830, 1879267, 1185829, 480581, 1872226, 1178571, 1872270, 1872271, 1178572] 22
[1769592, 401

In [88]:
K/len(C.open_paths)

0.17571480275063336

In [85]:
for k in C.open_paths[2]:
    if C.open_paths[2].count(k) != 1 :
        print(k)

1343428
1343428


In [41]:
def sort(x,y):
    return (min(x,y),max(x,y))
sort(3,5)

(3, 5)

In [105]:
#detect cycles while detecting open_paths for com^plexity matters
def fill_open_paths_with_cycles(C,paths,layer):
    antecedant = dict()
    visited = set()
    set_paths = dict()
    index_paths = dict()
    cycles = []
    K = []
    for point in tqdm(layer):
        paths[point] = [point]
        antecedant[point] = point
        index_paths[point] = dict()
        set_paths[point] = dict()

    while layer != {-1}:

        new_layer = set()
        for point in (layer):
            
            if point != -1 :
                next_nodes = [n for n in C.Res_graph[point] if (not C.incycles[n] and n not in visited)]
                if next_nodes == []:
                    new_layer.add(-1)
                else:
                    next_node = next_nodes[0]
                    visited.add(next_node)

                    if next_node in set_paths[antecedant[point]]:
                        ind = index_paths[antecedant[point]][next_node]
                        cycle = paths[ind:].append(next_node)
                        cycles.append(cycle)
                        point = paths[ind-1]
                    
                    else :
                        next_nodes = [n for n in C.Res_graph[point] if (not C.incycles[n] and n not in visited)]
                        paths[antecedant[point]].append(next_node)
                        set_paths[antecedant[point]].add(next_node)
                        index_paths[antecedant[point]][next_node] = len(paths[antecedant[point]]) - 1
                        new_layer.add(next_node)
                        antecedant[next_node] = antecedant[point]


        assert layer != new_layer, ("there is a repetition",layer,new_layer)
        layer = new_layer
        

    C.open_paths = list(paths.values())
    C.cycles = cycles
    return K,antecedant
    

In [25]:
C.open_paths=[]
paths = dict()
layer = set(C.starting_open_paths)
fill_open_paths(C,paths,layer)

TypeError: fill_open_paths() takes 1 positional argument but 3 were given

In [ ]:
def detect_cycles_unitaire(C):
    Unitaire = set()
    for point in C.Res_graph:
        if len(C.Res_graph[point]) == 1:
            Unitaire.add(point)
    paths = dict()
    paths_set = dict()
    pass

In [109]:
Closed_points = set()
Open_points = set()

for path in C.open_paths:
    for point in path:
            C.incycles[point] = True
            Open_points.add(point)

for path in C.cycles:
    for point in path:
            C.incycles[point] = True
            Closed_points.add(point)

In [56]:
Unitaire = set()
for point in C.Res_graph:
    if C.incycles[point] == False: 
        if len(C.Res_graph[point]) == 1:
            Unitaire.add(point)
Secondaire = deepcopy(Unitaire)

visited_edges = set()
for point in Unitaire :
    next_point = [n for n in C.Res_graph[point] if (min(point,n),max(n,point)) not in visited_edges]



In [110]:
Not_in_cycles = []
for node in range(len(C.mapping)):
    if not C.incycles[node] :
        Not_in_cycles.append(node)
len(Not_in_cycles)

474934

In [19]:
True in C.end_of_path

True

In [22]:
node = 10
boolean = C.Res_graph[10][0] in Not_in_cycles
while  boolean :
    print(node)
    Not_in_cycles.remove(node)
    boolean = False
    for k in C.Res_graph[node]:
        if k in Not_in_cycles:
            boolean = True
            node = k



10
612531
1333919
1333920
1333921
37
6256
1345003
6257
38
612560
1333944
612585
90
1338584
617711
1338617
118
1333980
612608
1333962
1333961
612607
612638
612665
1334029
612664
612637
117
1338614
617709
617676
6288
624434
6254
617612


In [24]:
C.Res_graph[617612]

[10, 1338500]

In [104]:
C.Res_graph[10] in Not_in_cycles

False

In [83]:
for op in C.open_paths:
    if 617565 in op:
        print(op)

[612499, 1333876, 0, 617565]


In [92]:
actual_starting_open_paths = []
actual_ending_points = []
for op in C.open_paths:
    actual_starting_open_paths.append(op[0])
    actual_ending_points.append(op[-1])

In [89]:
starting_open_paths_left = []
ending_pointsè
for point in C.starting_open_paths:
    if point not in actual_starting_open_paths:
        starting_open_paths_left.append(point)



In [1]:
len(C.starting_open_paths) - len(actual_starting_open_paths) 

NameError: name 'C' is not defined

In [94]:
for op in C.cycles:
    if 1933316 in op:
        print(op)

1933316


AttributeError: 'Resiuals' object has no attribute 'end_of_path'

In [35]:
for op in C.open_paths:
    if 1248021 in op:
        print(op)

[1933316, 1241083, 529714, 1248021, 529743, 523860, 1234296, 1926622]


In [32]:
C.end_of_path[1926622]

True

In [ ]:
for k in tqdm(range(C.list_res)):

In [ ]:
for k in Begin :
    if k not in C.visited:
        print(k)

In [ ]:
my_set = {1, 2, 3, 4, 5, 6}
my_list = [2, 4, 9]

my_set.difference_update(my_list)
print(my_set)

{1, 3, 5, 6}


In [ ]:
len(Begin)

16578

In [ ]:
len(C.linear_paths)

10727

In [ ]:
set(C.dict_open_paths.values())

{0, 1}

In [ ]:
for op in C.open_paths:
    if 58329 in op:
        print(op)

[58329, 126712, 193867]
[58329, 126712, 193867]
[58329, 126712, 193867]


In [ ]:
len(C.open_paths)/3

38.0

In [ ]:
C.starting_open_paths[123]

63433

In [ ]:
C.Res_graph[63433]

[63424, 4891]

In [ ]:
for adjacency in C.Res_graph.values():
    if 4891 in adjacency :
        print(adjacency)

[63424, 4891]
[63424, 4891]


In [43]:
Poin
(list(paths.values()))

[[1933316, 1241046, 1933276],
 [1998855,
  583442,
  1992222,
  1301849,
  1301820,
  577199,
  1985568,
  577198,
  1301789,
  583351,
  1998765],
 [1343496,
  4737,
  616341,
  616311,
  4671,
  10916,
  1349914,
  10915,
  1343430,
  1343429,
  4670,
  616276,
  4639,
  1343391,
  4640,
  1337341,
  616250,
  1337314,
  4606,
  10858,
  1349848,
  629410,
  629448,
  17007,
  636174,
  636138,
  1356254,
  23069,
  1362749,
  642909,
  1362785,
  23095,
  636173,
  1356313,
  636172,
  1356280,
  636137,
  23068,
  1362747,
  642908,
  1362784,
  1362783,
  23094,
  16965,
  629407,
  629373,
  1349814,
  16908,
  1356223,
  636080,
  16871,
  629318,
  10763,
  622526,
  10731,
  629285,
  16839,
  1356157,
  16838,
  1349752,
  1349753,
  629319,
  629343,
  1349815,
  629374,
  1349845,
  10855,
  4604,
  1337310,
  1337311,
  616218,
  4576,
  622599,
  1343284,
  10793,
  1349781,
  629344,
  10821,
  622630,
  10857,
  1349847,
  629409,
  629447,
  10913,
  1343427,
  4669,
 

In [ ]:
C.incycles[129011]

False

In [ ]:
J = [len(cycle) for cycle in C.cycles]

In [ ]:
max(J)

89

In [ ]:
C.cycles = []
C.incycles = [1]*len(C.mapping)
C.visited = [1]*len(C.mapping)
C.dfs_iterative(613037)

In [ ]:
cpt

58

In [ ]:
for cycle in C.cycles:
    if 1334354 in cycle:
        print(len(cycle))

7416


In [ ]:
for cycle in C.cycles:
    if 613037 in cycle:
        print(cycle)

[613018, 613037, 1334354, 634, 6888, 1345645, 1345646, 613063, 1334377, 1334378, 613064, 1334321, 613017, 1334340, 613018]
[613037, 1334354, 634, 6888, 1345645, 1345646, 613063, 1334377, 1334378, 613064, 1334321, 613017, 1334340, 613037]


In [ ]:
C.visited

[]

In [ ]:
len(np.where(np.array(C.incycles) == 1)[0])

606743

In [ ]:
for open_path in C.open_paths:
    assert C.is_boundary(open_path[0]), "you hav error"

In [ ]:
L = [len(C.Res_graph[node]) for node in C.starting_open_paths]
L

[1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 2,
 1,
 2,
 3,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 2,
 2,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 2,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 1,
 1,
 2,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 2,
 2,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 2,
 1,
 1,
 1,


In [ ]:
incycles = [1]*len(C.mapping)
def detect_cycles(C):
    path_indices = {}
    for v in tqdm(range((len(C.mapping)))):
        if C.connex[v] != -1:
            dfs_iterative(C,v,path_indices)

def dfs_iterative(C, start,path_indices):
    stack = [(start, -1)]
    path_indices[start] = 0
    path = [start]
    while stack:
        node, parent = stack.pop()
        if C.connex[node] != -1:
            for neighbour in C.Res_graph[node]:
                if neighbour != parent:
                    if neighbour in path:
                        cycle_start_index = path.index(neighbour)
                        C.cycles.append(path[cycle_start_index:] + [neighbour])
                        for point in path[cycle_start_index:] + [neighbour]:
                            C.connex[point] = -1
                            C.incycles[point] = -1
                    else:
                        stack.append((neighbour, node))
                        path = path + [neighbour]
                        path_indices[neighbour] = len(path)
                    

            C.connex[node] = -1



In [ ]:
C.cycles = []
for point in range(len(C.mapping)):
    C.connex[point] = 2

detect_cycles(C)

100%|██████████| 2023266/2023266 [00:31<00:00, 64708.27it/s]  


In [ ]:
len(np.where(np.array(incycles) == 1)[0])/len(C.mapping)

0.3310459425503122

In [ ]:
669794/len(C.mapping)

0.3310459425503122

In [ ]:
L = [len(compo) for compo in C.connected_components.values()]
bad = L.index(1)
colour = list(C.connected_components.keys())[bad]
C.connected_components[colour]

[28]

In [ ]:
C.Res_graph[28]

[1338522, 1338523]

In [ ]:
from tqdm import tqdm
for r in tqdm(C.list_res):
    C.Res_graph[R[r]] = []

  0%|          | 0/6069798 [00:00<?, ?it/s]

100%|██████████| 6069798/6069798 [00:09<00:00, 632648.60it/s] 


In [ ]:
R

[0]

In [ ]:
for res in tqdm(C.list_res):
    i,j,k,axe,value = res
    # if i not in [0,X-1] and j not in [0,Y-1] and k not in [0,Z-1]:
    C.nodes_of_not_boundary(res)

  0%|          | 0/2023266 [00:00<?, ?it/s]


KeyError: 0

In [ ]:
C.identify_connected_component()

In [ ]:
D = (C.connected_components).values()
Number = [len(k) for k in D]
R = sorted(Number)
R

[0]

In [ ]:
R

[0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [ ]:
Number

[2,
 1781553,
 3,
 5,
 6,
 2,
 2,
 2,
 5,
 2,
 2,
 5,
 1,
 5,
 5,
 1,
 4,
 2,
 8,
 4,
 2,
 6,
 2,
 4,
 10,
 4,
 4,
 4,
 4,
 16,
 6,
 4,
 6,
 15,
 8,
 4,
 4,
 8,
 4,
 4,
 12,
 4,
 2,
 3,
 6,
 24,
 4,
 12,
 6,
 4,
 4,
 2,
 4,
 4,
 6,
 4,
 4,
 8,
 8,
 22,
 4,
 14,
 8,
 6,
 2,
 4,
 6,
 2,
 10,
 4,
 4,
 4,
 2,
 7,
 4,
 8,
 10,
 14,
 4,
 4,
 6,
 6,
 6,
 4,
 9,
 4,
 3,
 8,
 4,
 2,
 12,
 6,
 2,
 2,
 4,
 6,
 10,
 4,
 4,
 9,
 6,
 4,
 8,
 4,
 4,
 10,
 6,
 6,
 6,
 4,
 4,
 6,
 6,
 1,
 6,
 4,
 2,
 10,
 4,
 6,
 4,
 4,
 4,
 6,
 4,
 4,
 14,
 6,
 4,
 1,
 4,
 4,
 6,
 3,
 4,
 4,
 6,
 4,
 4,
 6,
 10,
 5,
 4,
 4,
 14,
 4,
 4,
 10,
 6,
 12,
 8,
 4,
 16,
 2,
 8,
 3,
 6,
 4,
 4,
 4,
 4,
 14,
 12,
 6,
 4,
 4,
 4,
 4,
 4,
 10,
 9,
 4,
 6,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 6,
 4,
 4,
 10,
 4,
 4,
 4,
 5,
 6,
 10,
 6,
 5,
 4,
 6,
 6,
 4,
 4,
 8,
 11,
 4,
 11,
 2,
 4,
 14,
 3,
 4,
 2,
 3,
 1,
 2,
 1,
 1,
 3,
 5,
 8,
 4,
 3,
 5,
 2,
 3,
 4,
 7,
 2,
 2,
 23,
 2,
 3,
 7,
 2,
 2,
 16,
 3,
 3,
 3,
 4,
 2,
 2,
 6,
 2,
 19,
 3

In [ ]:
0 in (set(list(C.connex.values())))

False

In [ ]:
R = []
for key in tqdm(C.Res_graph.keys()):
    if len(C.Res_graph[key]) not in [0,1,2,3]:
        R.append(key)

100%|██████████| 2023266/2023266 [00:00<00:00, 2402570.20it/s]
